这篇博客（[视频](https://www.bilibili.com/video/BV1r3411979W/?spm_id_from=333.999.0.0&vd_source=b4496ba547455f3db613a8f8de6bea01)）记录区块链技术投资公告效应研究样本的数据清洗过程，主要内容是剔除业绩预告、分红、收购并购、重组以及其他突发事件公告日落在事件窗口期之内样本公司。
<!-- TEASER_END -->

In [1]:
import logging
from datetime import date, timedelta

import numpy as np
import pandas as pd
import requests

logging.basicConfig(level=logging.INFO)

In [2]:
data = pd.read_csv('./data/tidyed_data.csv', converters={u'secCode':str})

In [3]:
data.head()

,secCode,secName,orgId,announcementTitle,announcementTime,adjunctUrl,announcementContent
0,002152,广电运通,9900003423,广电运通：2016年4月1日投资者关系活动记录表,2016-04-06,http://static.cninfo.com.cn/finalpage/2016-04-...,9、公司提过有储备区块链技术，是否可以介绍？
1,002177,御银股份,9900003781,御银股份：2016年4月21日投资者关系活动记录表,2016-04-21,http://static.cninfo.com.cn/finalpage/2016-04-...,4、公司在区块链技术上未来有什么规划？
2,300386,飞天诚信,9900023058,飞天诚信：2016年4月22日投资者关系活动记录表,2016-04-25,http://static.cninfo.com.cn/finalpage/2016-04-...,公司在区块链部分如何布局?
3,000997,新 大 陆,gssz0000997,新 大 陆：董事会关于本次募集资金使用的可行性分析,2016-06-03,http://static.cninfo.com.cn/finalpage/2016-06-...,支付行业技术创新和业务创新不断涌现，包括 NFC、条码（二维码）支付、可信服务平台（TSM）...
4,300423,鲁亿通,9900028812,鲁亿通：关于本次交易前12个月内购买、出售资产的说明,2016-06-09,http://static.cninfo.com.cn/finalpage/2016-06-...,同时，上市公司拟采用定价发行的方式向纪法清、孔剑平、孙奇锋、王麒诚和中信建投-数贝泽华人工智...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   secCode              394 non-null    object
 1   secName              394 non-null    object
 2   orgId                394 non-null    object
 3   announcementTitle    394 non-null    object
 4   announcementTime     394 non-null    object
 5   adjunctUrl           394 non-null    object
 6   announcementContent  394 non-null    object
dtypes: object(7)
memory usage: 21.7+ KB


In [5]:
def get_json(codedata, seDate):
    """获取 json 数据

    Args:
        codedata (str): 'code,orgId'
        seDate (str): '2019-05-03~2019-05-05'

    Returns:
        res.json(): json data
    """
    url = 'http://www.cninfo.com.cn/new/hisAnnouncement/query'
    headers = {
        "Accept-Encoding": "gzip",
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
    }
    
    payload = {
#       'stock': 'code,orgId',
        'tabName': 'fulltext',
        'pageSize': '30',
        'pageNum': '1',
        'column': 'szse',
        'category': 'category_yjygjxz_szsh;category_qyfpxzcs_szsh;category_zf_szsh;category_jj_szsh;category_tbclts_szsh;category_tszlq_szsh;category_ndbg_szsh;category_bndbg_szsh;category_yjdbg_szsh;category_sjdbg_szsh;category_sf_szsh;category_pg_szsh;category_gszq_szsh;category_kzzq_szsh;category_gqjl_szsh;category_gqbd_szsh;category_cqdq_szsh;category_qtrz_szsh;category_zj_szsh;',
        'plate': 'sz',
#       'seDate': '2019-05-03~2019-05-28',
        'isHLtitle': 'true',
    }
    payload['stock'] = codedata
    payload['seDate'] = seDate
    
    try:
        res = requests.post(url=url, headers=headers, params=payload)
        return res.json()
    except:
        return

In [6]:
get_json('300427,9900023902', '2019-05-07~2019-05-09')['totalAnnouncement']

0

In [7]:
def filter_event(codedata, seDate):
    data = get_json(codedata, seDate)
    if data is not None:
        if data['totalAnnouncement'] > 0:
            return False
        return True
    else:
        logging.info(codedata)
        return False

In [8]:
codedata = '300427,9900023902'
seDate = '2019-05-07~2019-05-09'
filter_event(codedata, seDate)

True

In [9]:
data.head()

,secCode,secName,orgId,announcementTitle,announcementTime,adjunctUrl,announcementContent
0,002152,广电运通,9900003423,广电运通：2016年4月1日投资者关系活动记录表,2016-04-06,http://static.cninfo.com.cn/finalpage/2016-04-...,9、公司提过有储备区块链技术，是否可以介绍？
1,002177,御银股份,9900003781,御银股份：2016年4月21日投资者关系活动记录表,2016-04-21,http://static.cninfo.com.cn/finalpage/2016-04-...,4、公司在区块链技术上未来有什么规划？
2,300386,飞天诚信,9900023058,飞天诚信：2016年4月22日投资者关系活动记录表,2016-04-25,http://static.cninfo.com.cn/finalpage/2016-04-...,公司在区块链部分如何布局?
3,000997,新 大 陆,gssz0000997,新 大 陆：董事会关于本次募集资金使用的可行性分析,2016-06-03,http://static.cninfo.com.cn/finalpage/2016-06-...,支付行业技术创新和业务创新不断涌现，包括 NFC、条码（二维码）支付、可信服务平台（TSM）...
4,300423,鲁亿通,9900028812,鲁亿通：关于本次交易前12个月内购买、出售资产的说明,2016-06-09,http://static.cninfo.com.cn/finalpage/2016-06-...,同时，上市公司拟采用定价发行的方式向纪法清、孔剑平、孙奇锋、王麒诚和中信建投-数贝泽华人工智...


In [10]:
final_code = []
for code,orgId,time in zip(data.loc[:, 'secCode'], data.loc[:, 'orgId'], data.loc[:, 'announcementTime']):
    codedata = code + ',' + orgId
    # [-1,1]
    start = (date.fromisoformat(time) - timedelta(days=1)).strftime('%Y-%m-%d')
    end = (date.fromisoformat(time) + timedelta(days=1)).strftime('%Y-%m-%d')
    sedate = start + '~' + end
    if filter_event(codedata, seDate):
        final_code.append(code)

In [11]:
len(final_code)

330

In [12]:
def filter_code(c):
    if c in final_code:
        return True
    return False

In [13]:
final = data.loc[data.loc[:, 'secCode'].apply(lambda c: filter_code(c)), :]

In [14]:
final

,secCode,secName,orgId,announcementTitle,announcementTime,adjunctUrl,announcementContent
0,002152,广电运通,9900003423,广电运通：2016年4月1日投资者关系活动记录表,2016-04-06,http://static.cninfo.com.cn/finalpage/2016-04-...,9、公司提过有储备区块链技术，是否可以介绍？
1,002177,御银股份,9900003781,御银股份：2016年4月21日投资者关系活动记录表,2016-04-21,http://static.cninfo.com.cn/finalpage/2016-04-...,4、公司在区块链技术上未来有什么规划？
2,300386,飞天诚信,9900023058,飞天诚信：2016年4月22日投资者关系活动记录表,2016-04-25,http://static.cninfo.com.cn/finalpage/2016-04-...,公司在区块链部分如何布局?
4,300423,鲁亿通,9900028812,鲁亿通：关于本次交易前12个月内购买、出售资产的说明,2016-06-09,http://static.cninfo.com.cn/finalpage/2016-06-...,同时，上市公司拟采用定价发行的方式向纪法清、孔剑平、孙奇锋、王麒诚和中信建投-数贝泽华人工智...
5,000961,中南建设,gssz0000961,中南建设：关于本公司投资上海承泰信息科技股份有限公司的进展公告,2016-06-23,http://static.cninfo.com.cn/finalpage/2016-06-...,，以区块链技术处理玛娜花园不同版本的个人数据，形成对数据信息的安全保护。
...,...,...,...,...,...,...,...
388,300133,华策影视,9900013428,华策影视：关于全资子公司增资扩股暨关联交易的公告,2021-12-17,http://static.cninfo.com.cn/finalpage/2021-12-...,影业上海基于节目库搭建的正版影视素材库“华策新视界”已进入正式运营，目前正在推进基于区块链技...
389,300235,方直科技,9900011849,方直科技：关于对深圳证券交易所关注函的回复公告,2021-12-20,http://static.cninfo.com.cn/finalpage/2021-12-...,元宇宙是整合多种新技术而产生的新型虚实相融的互联网应用和社会形态，它基于扩展现实技术提供沉浸...
390,300649,杭州园林,9900023811,杭州园林：关于收购参股公司部分股权的公告,2021-12-24,http://static.cninfo.com.cn/finalpage/2021-12-...,一般项目：网络技术服务；物联网技术服务；技术服务、技术开发、技术咨询、技术交流、技术转让、技...
391,300533,冰川网络,9900027730,冰川网络：关于全资子公司设立三家子公司并完成工商注册登记的公告,2021-12-24,http://static.cninfo.com.cn/finalpage/2021-12-...,人工智能通用应用系统；智能控制系统集成；量子计算技术服务；电竞信息科技；云计算装备技术服务；...


In [15]:
final.to_csv('./data/final.csv', index=False, encoding='utf_8_sig')